In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:


# Read the token from the file
token_path = '/content/drive/MyDrive/github_token.txt'
with open(token_path, 'r') as token_file:
    github_token = token_file.read().strip()

%cd /content/drive/MyDrive/llm//

# Define the GitHub repository URL with the token for authentication
repo_url = f"https://{github_token}@github.com/nsharma4/llm.git"


# Update the remote URL to include the token
!git -C /content/drive/MyDrive/llm/ remote set-url origin {repo_url}

!git config --global user.email "nealsharma99@gmail.com"
!git config --global user.name "nsharma4"




/content/drive/.shortcut-targets-by-id/18XarxB4-UrU23qBHLzKLjjeavqd8T6Js/llm


In [3]:

!pip install -r requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.5/602.5 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.

In [4]:
import os
import pandas as pd
import langchain
import nltk
# from kaggle.api.kaggle_api_extended import KaggleApi
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.docstore.document import Document
from nltk.corpus import wordnet
import torch

# Ensure NLTK resources are downloaded
nltk.download('wordnet')

# # Authenticate with the Kaggle API
# api = KaggleApi()


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [13]:

# # Download the "All The News" dataset from Kaggle
# api.dataset_download_files('asad1m9a9h6mood/news-articles', path='data/', unzip=True)

# Load and process the primary dataset (All The News)
file_path = 'data/Articles.csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1')

document_list_primary = [
    Document(
        page_content=row['Article'],
        metadata={'date': row['Date'], 'heading': row['Heading'], 'news_type': row['NewsType']}
    )
    for _, row in df.iterrows()
]

# Load and process the secondary dataset (News Category Dataset)
secondary_file_path = 'data/News_Category_Dataset_v3.json'
df_secondary = pd.read_json(secondary_file_path, lines=True)

document_list_secondary = [
    Document(
        page_content=row['headline'],
        metadata={'headline': row['headline'], 'short_description': row['short_description']}
    )
    for _, row in df_secondary.iterrows()
]


In [14]:
# NOTE: Truncate document lists to control size
max_length = 1000
if len(document_list_primary) > max_length:
    document_list_primary = document_list_primary[:max_length]
if len(document_list_secondary) > max_length:
    document_list_secondary = document_list_secondary[:max_length]


In [15]:
# Initialize SentenceTransformer and create vector stores for both datasets
embedding_function = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')

# Vector store for primary dataset (All The News)
vector_store_primary = Chroma.from_documents(document_list_primary, embedding_function)

# Vector store for secondary dataset (News Category Dataset)
vector_store_secondary = Chroma.from_documents(document_list_secondary, embedding_function)



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
# Expand query using WordNet
def expand_query(query):
    expanded_terms = []
    for word in query.split():
        synsets = wordnet.synsets(word)
        if synsets:
            expanded_terms.extend([lemma.name() for lemma in synsets[0].lemmas()])
    return " ".join(set(query.split() + expanded_terms))

# Multi-source dynamic retrieval
def dynamic_retrieval(query, docs, vector_store_primary, vector_store_secondary, k=2):
    expanded_query = expand_query(query)

    # Retrieve documents from the primary dataset (All The News)
    new_docs_primary = vector_store_primary.similarity_search(expanded_query, k=k)

    # Retrieve documents from the secondary dataset (News Category Dataset)
    new_docs_secondary = vector_store_secondary.similarity_search(expanded_query, k=k)

    # Merge results from both datasets, ensuring no duplicates
    retrieved_docs = {doc.metadata.get('heading') or doc.metadata.get('headline') for doc in docs}

    unique_new_docs_primary = [doc for doc in new_docs_primary if doc.metadata.get('heading') not in retrieved_docs]
    unique_new_docs_secondary = [doc for doc in new_docs_secondary if doc.metadata.get('headline') not in retrieved_docs]

    return unique_new_docs_primary + unique_new_docs_secondary


In [18]:
# Set the correct model path
model_path = "/content/drive/MyDrive/llm/Mistral-7B-v0.1"

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
# Expand the initial query
query = "Tell me about what the news in US politics is about"

# Initial document retrieval using both datasets
current_docs = dynamic_retrieval(query, [], vector_store_primary, vector_store_secondary, k=2)

# Use both headline and short_description for the context string
context = " ".join([doc.page_content for doc in current_docs])

# Caching tokenized context to avoid re-tokenization
cached_inputs = tokenizer(context, return_tensors="pt")

In [22]:
max_iterations = 1
for i in range(max_iterations):
    # Generate text
    with torch.no_grad():
        outputs = model.generate(
            cached_inputs['input_ids'],
            max_new_tokens=40,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            length_penalty=1.5
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Iteration {i+1} generated text:\n{generated_text}\n")

    # Check if more information is needed
    if len(generated_text.split()) < 20 or "need more information" in generated_text.lower():
        new_docs = dynamic_retrieval(generated_text, current_docs, vector_store, k=2)
        if new_docs:
            current_docs += new_docs
            new_context = " ".join([doc.page_content for doc in new_docs])
            context += " " + new_context
            cached_inputs = tokenizer(context, return_tensors="pt")
        else:
            print("No new relevant information found. Ending generation.")
            break
    else:
        print("Sufficient information generated. Ending generation.")
        break

# # Cell 10: Clean up (optional)
del model
torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:634: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `1.5` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu

Iteration 1 generated text:
Mar-A-Lago Search Affidavit: Trump Was Insecurely Storing Most Sensitive Intel At Club OOPS! Reporters Spot What Sen. Ted Cruz Was Looking At On His Phone During Hearings Mar-A-Lago Search Affidavit: Trump Was Insecurely Storing Most Sensitive Intel At Club OOPS! Reporters Spot What Sen. Ted Cruz Was Looking At On His Phone During Hearings

Republican senators have been trying to discredit the FBI’s search of Donald Trump’s Mar-a-Lago resort, which turned up dozens of top secret documents that

Sufficient information generated. Ending generation.


In [24]:
# !git -C /content/drive/MyDrive/llm/ add .

# !git -C /content/drive/MyDrive/llm/ commit -m "cleared outputs"
# !git push



[master 37b449a] cleared outputs
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite 2_4.ipynb (99%)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 96 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 468 bytes | 117.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/nsharma4/llm.git
   dea9b40..37b449a  master -> master
